In [1]:
# !pip install pandas
# !pip install spacy
# !pip install -U pip setuptools wheel
# !python -m spacy download en_core_web_sm

In [125]:
import pandas as pd
import numpy as np
import spacy
import csv
pd.set_option('display.max_colwidth', None)

In [126]:
raw_df = pd.read_csv ('../datasets/un-general-debates-2000s.csv')

In [127]:
raw_df.shape

(1491, 4)

In [186]:
# Converting paragraphs to one long line.
# Removing what looks like page/paragraph numbers.
# Replacing tabs with space.
raw_df ['processed'] = raw_df['text'].replace(r"\r\n", " ", regex=True) \
                                        .replace(r"^[0-9]+\r\n\r\n\r\n","", regex=True) \
                                        .replace(r"\t"," ", regex=True)



In [187]:
raw_df.head()

session  year country  \
0       55  2000     IND   
1       55  2000     VCT   
2       55  2000     LKA   
3       55  2000     ISL   
4       55  2000     NLD   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [188]:
import spacy
from spacy.lang.en import English
nlp = English()
nlp.add_pipe('sentencizer')

def extract_sentences(s):
    doc = nlp(s)
    return [sent.text for sent in doc.sents]

sentences_list = [ np.array(extract_sentences(x)) for x in raw_df['processed']]

In [189]:
sentences_list[1]

array(['I take this opportunity to congratulate the President of the General Assembly on his election.',
       'I am confident that his vision and energy will guide this session to a successful conclusion.',
       'I also thank Mr. Theo-Ben Gurirab, who led the fifty-fourth session of the General Assembly with distinction and efficiency.',
       'I commend Mr. Kofi Annan, the esteemed Secretary-General of this Organization, for the thoughtful recommendations which he has put forward in his report to the Millennium Summit entitled “We the peoples: the role of the United Nations in the twenty-first century”.',
       'His vision and stewardship have proved to be vital factors in advancing the work and mission of the United Nations at this critical time in the history of the Organization and the world.',
       'I join the previous speakers in welcoming the nation of Tuvalu as the 189th and newest Member of the United Nations family.',
       'Our newest entrant is also the smallest Me

In [161]:
final = [i for s in sentences_list for i in s]

In [162]:
len(final)

159645

In [163]:
final

['India congratulates Mr. Harri Holkeri on his election as President of the first General Assembly of this millennium.',
 'We are encouraged that he intends to take the Millennium Declaration as the mandate for the work of the Organization during his presidency.',
 'I take this opportunity to sincerely compliment and to place on record our high appreciation of Foreign Minister Mr. Theo-Ben Gurirab of Namibia for his guidance of the General Assembly in the past year, which included three special sessions and, of course, meticulous preparations for and a successful conclusion to the Millennium Summit.',
 "I join India's voice to those of other delegations in noting with appreciation the dedication with which Secretary-General Kofi Annan has served the United Nations at a critical point.",
 'I am delighted on this occasion to warmly welcome Tuvalu as the newest Member of the United Nations.',
 'Just 10 days ago, almost 150 heads of State and Government assembled here for the Millennium Su

In [164]:
sentences_df = pd.DataFrame(final, columns = ['text'])
sentences_df['label'] = '0'

In [180]:
sentences_df.head()


,text,label
0,India congratulates Mr. Harri Holkeri on his election as President of the first General Assembly of this millennium.,0
1,We are encouraged that he intends to take the Millennium Declaration as the mandate for the work of the Organization during his presidency.,0
2,"I take this opportunity to sincerely compliment and to place on record our high appreciation of Foreign Minister Mr. Theo-Ben Gurirab of Namibia for his guidance of the General Assembly in the past year, which included three special sessions and, of course, meticulous preparations for and a successful conclusion to the Millennium Summit.",0
3,I join India's voice to those of other delegations in noting with appreciation the dedication with which Secretary-General Kofi Annan has served the United Nations at a critical point.,0
4,I am delighted on this occasion to warmly welcome Tuvalu as the newest Member of the United Nations.,0


In [181]:
# There are sentences that ended up with only what looks like page numbers.
# Remove paragrah indendations.
# Remove multiple spaces with a single space. Doesn't help much but we will have a cleaner dataset.
sentences_df ['text'] = sentences_df['text'].replace(r"([0-9]+).","", regex=True) \
                                        .replace(r"^\t","", regex=True) \
                                        .replace(r"[ ]+"," ", regex=True)

In [190]:
# Remove sentences that got fewer than 10 characters. No logic here just to filter out some really small sentences.
sentences_df.drop(sentences_df[sentences_df['text'].map(len) <= 10].index, inplace=True)

In [193]:
# Write to a CSV file
sentences_df.to_csv('../datasets/un-data-processed.csv', quotechar='"',quoting=csv.QUOTE_NONNUMERIC, index=False)

In [191]:
# Write to a text file to load to docanno for labelling.
sentences_df['text'].to_csv('../datasets/un-data-processed.txt', quotechar='"',quoting=csv.QUOTE_NONNUMERIC, index=False)

In [184]:
sentences_df.head()

,text,label
0,India congratulates Mr. Harri Holkeri on his election as President of the first General Assembly of this millennium.,0
1,We are encouraged that he intends to take the Millennium Declaration as the mandate for the work of the Organization during his presidency.,0
2,"I take this opportunity to sincerely compliment and to place on record our high appreciation of Foreign Minister Mr. Theo-Ben Gurirab of Namibia for his guidance of the General Assembly in the past year, which included three special sessions and, of course, meticulous preparations for and a successful conclusion to the Millennium Summit.",0
3,I join India's voice to those of other delegations in noting with appreciation the dedication with which Secretary-General Kofi Annan has served the United Nations at a critical point.,0
4,I am delighted on this occasion to warmly welcome Tuvalu as the newest Member of the United Nations.,0


In [192]:
raw_df ['processed'].to_csv('../datasets/un-data-processed-unsplit.txt', quotechar='"',quoting=csv.QUOTE_NONNUMERIC, index=False)